In [49]:
import pandas as pd
import matplotlib.pyplot as plt

In [50]:
df = pd.read_csv("parsed_logs_fi.csv")

In [51]:
df["config"].unique()

array(['c10_res44_test_01_bn-relu_base', 'c10_res44_test_02_bn-relu6',
       'c10_res44_test_02_bn-relu6_base', 'c10_res44_test_02_relu6-bn',
       'c10_res44_test_02_relu6-bn_nanfilter',
       'c10_res44_test_02_gelu6_nans'], dtype=object)

In [52]:
configs = dict(
    # Baseline C100
    BaseC100="c100_res44_test_01_bn-relu_base.yaml",
    # "c100_res44_test_02_bn-relu6.yaml",
    # Order inversion with relu6 C100
    OrderInvC100="c100_res44_test_02_relu6-bn.yaml",
    # "c100_res44_test_02_bn-relu6_base.yaml",
    BaseBnRelu6C100="c100_res44_test_02_bn-relu6_base.yaml",
    BnRelu6C100="c100_res44_test_02_bn-relu6.yaml",
    # Order inversion + nan filter + Relu6
    OrderINanRelu6C100="c100_res44_test_02_relu6-bn_nanfilter.yaml",
    # Gelu and nan C100
    GeluNanC100="c100_res44_test_02_gelu6_nans.yaml",

    # Baseline C10
    BaseC10="c10_res44_test_01_bn-relu_base.yaml",
    # "c10_res44_test_02_bn-relu6.yaml",
    BnRelu6C10="c10_res44_test_02_bn-relu6.yaml",
    # Order inversion with relu6 C10
    OrderInvC10="c10_res44_test_02_relu6-bn.yaml",
    # "c10_res44_test_02_bn-relu6_base.yaml",
    BaseBnRelu6C10="c10_res44_test_02_bn-relu6_base.yaml",
    OrderINanRelu6C10="c10_res44_test_02_relu6-bn_nanfilter.yaml",
    # Gelu and nan C10
    GeluNanC10="c10_res44_test_02_gelu6_nans.yaml",

    MobileNetC100="cifar100_mobilenetv2_x1_4.yaml",
    MobileNetC10="cifar10_mobilenetv2_x1_4.yaml"
)
configs_inv = {v.replace(".yaml", ""): k for k, v in configs.items()}

In [53]:
configs_df = pd.DataFrame([configs]).T.reset_index().rename(columns={0:"YAML", "index": "config"})

In [54]:
configs_df

,config,YAML
0,BaseC100,c100_res44_test_01_bn-relu_base.yaml
1,OrderInvC100,c100_res44_test_02_relu6-bn.yaml
2,BaseBnRelu6C100,c100_res44_test_02_bn-relu6_base.yaml
3,BnRelu6C100,c100_res44_test_02_bn-relu6.yaml
4,OrderINanRelu6C100,c100_res44_test_02_relu6-bn_nanfilter.yaml
5,GeluNanC100,c100_res44_test_02_gelu6_nans.yaml
6,BaseC10,c10_res44_test_01_bn-relu_base.yaml
7,BnRelu6C10,c10_res44_test_02_bn-relu6.yaml
8,OrderInvC10,c10_res44_test_02_relu6-bn.yaml
9,BaseBnRelu6C10,c10_res44_test_02_bn-relu6_base.yaml


In [55]:
df["config"] = df["config"].apply(lambda x: configs_inv[x])

In [56]:
df["count"] = 1

In [57]:
df[df["sdc"] == 0]

,config,ecc,hostname,fi_model,group,has_end,it,ker_time,acc_time,ker_err,acc_err,sdc,critical_sdc,count
0,BaseC10,OFF,caroltitaninria,FLIP_SINGLE_BIT,fp32,1,0,0.0,0.0,0,0,0.0,0.0,1
1,BaseC10,OFF,caroltitaninria,FLIP_SINGLE_BIT,fp32,1,0,0.0,0.0,0,0,0.0,0.0,1
4,BaseC10,OFF,caroltitaninria,FLIP_SINGLE_BIT,fp32,1,0,0.0,0.0,0,0,0.0,0.0,1
5,BaseC10,OFF,caroltitaninria,FLIP_SINGLE_BIT,fp32,1,0,0.0,0.0,0,0,0.0,0.0,1
6,BaseC10,OFF,caroltitaninria,FLIP_SINGLE_BIT,fp32,1,0,0.0,0.0,0,0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11993,GeluNanC10,OFF,caroltitaninria,WARP_ZERO_VALUE,fp32,1,0,0.0,0.0,0,0,0.0,0.0,1
11995,GeluNanC10,OFF,caroltitaninria,WARP_ZERO_VALUE,fp32,1,0,0.0,0.0,0,0,0.0,0.0,1
11997,GeluNanC10,OFF,caroltitaninria,WARP_ZERO_VALUE,fp32,1,0,0.0,0.0,0,0,0.0,0.0,1
11998,GeluNanC10,OFF,caroltitaninria,WARP_ZERO_VALUE,fp32,1,0,0.0,0.0,0,0,0.0,0.0,1


In [58]:
grouped_df = df.groupby(["hostname", "config"]).sum()

In [59]:
grouped_df[["sdc", "critical_sdc"]]

sdc  critical_sdc
hostname        config                                 
caroltitaninria BaseBnRelu6C10      958.0          15.0
                BaseC10             927.0         213.0
                BnRelu6C10          904.0          11.0
                GeluNanC10         1131.0          15.0
                OrderINanRelu6C10   882.0           2.0
                OrderInvC10         855.0           7.0

In [60]:
df.groupby(["hostname", "config", "fi_model"]).sum()

has_end   it    ker_time  \
hostname        config            fi_model                                      
caroltitaninria BaseBnRelu6C10    FLIP_SINGLE_BIT        400  183  354.415463   
                                  RANDOM_VALUE           400  262  502.137819   
                                  WARP_RANDOM_VALUE      400  367  666.952157   
                                  WARP_ZERO_VALUE        400  386  653.071643   
                                  ZERO_VALUE             400  226  481.183188   
                BaseC10           FLIP_SINGLE_BIT        400  202  408.849157   
                                  RANDOM_VALUE           400  252  429.089707   
                                  WARP_RANDOM_VALUE      400  325  653.486801   
                                  WARP_ZERO_VALUE        400  338  649.398939   
                                  ZERO_VALUE             400  253  443.084048   
                BnRelu6C10        FLIP_SINGLE_BIT        400  181  377.048041   
                                  RANDOM_VALUE           400  240  414.923168   
                                  WARP_RANDOM_VALUE      400  346  636.079970   
                                  WARP_ZERO_VALUE        400  377  670.749433   
                                  ZERO_VALUE             400  226  373.917111   
                GeluNanC10        FLIP_SINGLE_BIT        400  274  508.178343   
                                  RANDOM_VALUE           400  362  657.703108   
                                  WARP_RANDOM_VALUE      400  380  665.373041   
                                  WARP_ZERO_VALUE        400  362  599.143654   
                                  ZERO_VALUE             400  334  560.976321   
                OrderINanRelu6C10 FLIP_SINGLE_BIT        400  152  337.615886   
                                  RANDOM_VALUE           400  238  502.966534   
                                  WARP_RANDOM_VALUE      400  380  640.438115   
                                  WARP_ZERO_VALUE        400  349  625.013248   
                                  ZERO_VALUE             400  200  373.228641   
                OrderInvC10       FLIP_SINGLE_BIT        400  154  295.568260   
                                  RANDOM_VALUE           400  246  454.539728   
                                  WARP_RANDOM_VALUE      400  354  684.849478   
                                  WARP_ZERO_VALUE        400  301  572.232478   
                                  ZERO_VALUE             400  197  436.357054   

                                                        acc_time  ker_err  \
hostname        config            fi_model                                  
caroltitaninria BaseBnRelu6C10    FLIP_SINGLE_BIT     514.640994     1006   
                                  RANDOM_VALUE        727.653477     1781   
                                  WARP_RANDOM_VALUE   996.252061     2452   
                                  WARP_ZERO_VALUE    1020.190448     2510   
                                  ZERO_VALUE          666.152881     1569   
                BaseC10           FLIP_SINGLE_BIT     582.566222     1204   
                                  RANDOM_VALUE        667.749025     1667   
                                  WARP_RANDOM_VALUE   924.952548     2334   
                                  WARP_ZERO_VALUE     948.196635     2280   
                                  ZERO_VALUE          662.936247     3695   
                BnRelu6C10        FLIP_SINGLE_BIT     534.848697     1054   
                                  RANDOM_VALUE        640.361427     1561   
                                  WARP_RANDOM_VALUE   951.422627     2287   
                                  WARP_ZERO_VALUE    1019.010615     2492   
                                  ZERO_VALUE          593.917433     1446   
                GeluNanC10        FLIP_SINGLE_BIT     777.793715     1526   
                                  RANDOM_VALUE       1023.052883     2344 

In [61]:
grouped_df["sdc_rate"] = grouped_df["sdc"] / grouped_df["acc_time"]
grouped_df["critical_sdc_rate"] = grouped_df["critical_sdc"] / grouped_df["acc_time"]

In [62]:
grouped_df

has_end    it     ker_time     acc_time  \
hostname        config                                                       
caroltitaninria BaseBnRelu6C10        2000  1424  2657.760270  3924.889861   
                BaseC10               2000  1370  2583.908653  3786.400678   
                BnRelu6C10            2000  1370  2472.717722  3739.560800   
                GeluNanC10            2000  1712  2991.374466  4731.177808   
                OrderINanRelu6C10     2000  1319  2479.262423  3728.307887   
                OrderInvC10           2000  1252  2443.546998  3524.227989   

                                   ker_err  acc_err     sdc  critical_sdc  \
hostname        config                                                      
caroltitaninria BaseBnRelu6C10        9318     9318   958.0          15.0   
                BaseC10              11180    11180   927.0         213.0   
                BnRelu6C10            8840     8840   904.0          11.0   
                GeluNanC10           12123    12123  1131.0          15.0   
                OrderINanRelu6C10     9870     9870   882.0           2.0   
                OrderInvC10           8287     8287   855.0           7.0   

                                   count  sdc_rate  critical_sdc_rate  
hostname        config                                                 
caroltitaninria BaseBnRelu6C10      2000  0.244083           0.003822  
                BaseC10             2000  0.244824           0.056254  
                BnRelu6C10          2000  0.241740           0.002942  
                GeluNanC10          2000  0.239053           0.003170  
                OrderINanRelu6C10   2000  0.236568           0.000536  
                OrderInvC10         2000  0.242606           0.001986

In [63]:
grouped_df[["sdc", "has_end", "critical_sdc", "acc_time", "sdc_rate", "critical_sdc_rate"]]

sdc  has_end  critical_sdc     acc_time  \
hostname        config                                                          
caroltitaninria BaseBnRelu6C10      958.0     2000          15.0  3924.889861   
                BaseC10             927.0     2000         213.0  3786.400678   
                BnRelu6C10          904.0     2000          11.0  3739.560800   
                GeluNanC10         1131.0     2000          15.0  4731.177808   
                OrderINanRelu6C10   882.0     2000           2.0  3728.307887   
                OrderInvC10         855.0     2000           7.0  3524.227989   

                                   sdc_rate  critical_sdc_rate  
hostname        config                                          
caroltitaninria BaseBnRelu6C10     0.244083           0.003822  
                BaseC10            0.244824           0.056254  
                BnRelu6C10         0.241740           0.002942  
                GeluNanC10         0.239053           0.003170  
                OrderINanRelu6C10  0.236568           0.000536  
                OrderInvC10        0.242606           0.001986